## Installing the hooks
For installation, one needs to copy the hooks files from the 'hooks/' folder to the '.git/hooks/' folder. This can be done by executing the 'install-hooks.sh' Bash script.

Open Git Bash, and type in:

```bash
bash install-hooks.sh
```

To verify the installation has been successful, navigate to the '.git/hooks/' folder and check for our hook files, for example if the post-commit file is present.

## Post-commit hook

The post-commit hook is a hook that runs after a commit has been made. It can do various things after the commit, ranging from simply displaying a custom message if the commit was successful, running any post-commit tests and changes, automatically pushing, logging, notifications, etc. In this repository, we have a working post-commit script that displays a few messages after the commit. It also checks with the working directory if there are any additional changes that were not staged and warns the user about those. A log file is updated with a new entry, and a notification is sent to an email.

```bash
#!/bin/bash
# A Git post-commit hook that displays messages and additional information.


#---------------------#
#----- Flag check ----#
#---------------------#

# This check is necessary for our Logging part of code, where we amend the commit.
# Without this flag check, the post-commit hook would enter an infinite loop.
# Check if the commit has already been amended by looking for a flag file.
FILE_FLAG=".amend-flag"

if [ -f "$FILE_FLAG" ]; then
    # If the flag file exists, remove it and exit to avoid an infinite loop
    rm -f "$FILE_FLAG"
    echo "Exiting the re-triggered hook to avoid loop."
    exit 0
fi


#######################
####### Messages ######
#######################

echo "HOOK - POST-COMMIT - START:"
echo "Your commit succeeded. This is a message from IES squad."


#######################
####### Logging #######
#######################

echo "Logging commit details..."

# Log file location
FILE_LOG="hooks/commit-log.txt"

# Prepare the commit details
COMMIT_HASH=$(git log -1 --pretty=format:"%H")
COMMIT_MESSAGE=$(git log -1 --pretty=format:"%s")
COMMIT_AUTHOR=$(git log -1 --pretty=format:"%an")
COMMIT_DATE=$(git log -1 --pretty=format:"%ad")

# Append info to the log file
echo "------------------------------------------------" >> $FILE_LOG
echo "Commit hash: $COMMIT_HASH" >> $FILE_LOG
echo "Author: $COMMIT_AUTHOR" >> $FILE_LOG
echo "Date: $COMMIT_DATE" >> $FILE_LOG
echo "Message: $COMMIT_MESSAGE" >> $FILE_LOG
echo "" >> $FILE_LOG

echo "Done appending commit details to $FILE_LOG."

# Stage the log file so it gets included in the commit
git add $FILE_LOG

# Create the flag file to prevent re-triggering the hook
touch "$FILE_FLAG"

# Amend the last commit to include the updated log file, without changing the commit message
git commit --amend --no-edit

# Remove the flag file after the commit is amended
rm -f "$FLAG_FILE"

echo "Commit log updated and last commit amended."


#######################
######## Push #########
#######################

# Note: careful with this, more dangerous. Tested, works, but commenting it out for safer practices.
# Optionally, can automatically push the commit to the remote repository.
    # git push --force-with-lease origin $(git branch --show-current)


# Display the end message
echo "HOOK - POST-COMMIT - END"
```